# Pre-processing Idaho Observation Site stream data for WaDEQA upload.
Date Updated: 03/25/2022
Purpose:  To pre-process the Idaho os data into one master file for simple DataFrame creation and extraction


#### Idaho Nested Data Aqua API Data
The Aqua Info interface has an API behind it.  Here's a sample using “Pole Creek” site in the Upper Salmon Basin, using the GUID (99db207c15774d1c9a2f2a9daad85efa) in the URL, followed by the requsted time stamp.
- https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/99db207c15774d1c9a2f2a9daad85efa?fromDate=2020-09-21T14:33:47.134Z&toDate=2020-10-21T14:33:47.134Z

The GUID is inside each object’s “locDescription” property.
- https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/locations

In [1]:
import os
import numpy as np
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

#Working Directory and Input Files
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Idaho/ObservationSite/RawInputData"
os.chdir(workingDir)

In [2]:
# The general API search.  Returns a nestede data list with 228 entires.

url = "https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/locations"
responseList = json.loads(requests.get(url).text)

print(type(responseList))
print(len(responseList))
responseList

<class 'list'>
233


[{'locationName': 'Snake River nr Murphy, ID',
  'description': None,
  'identifier': '13172500',
  'uniqueId': '9b3cea8425b54d019abf4082e7058b4c',
  'locationType': 'Hydrology Station',
  'isExternalLocation': False,
  'latitude': 0.0,
  'longitude': 0.0,
  'srid': 4326,
  'elevation': '0',
  'elevationUnits': 'ft',
  'utcOffset': 0.0,
  'tags': [],
  'locationNotes': [],
  'locationRemarks': [],
  'referencePoints': [],
  'extendedAttributes': [{'name': 'PAKBUS', 'type': 'String', 'value': None},
   {'name': 'CHANNEL', 'type': 'String', 'value': None}],
  'locationDatum': {'referenceStandard': {'referenceStandard': None,
    'referenceStandardOffsets': []},
   'datumPeriods': []},
  'datasets': None,
  'locDescription': {'name': 'Snake River nr Murphy, ID',
   'identifier': '13172500',
   'uniqueId': '9b3cea8425b54d019abf4082e7058b4c',
   'isExternalLocation': False,
   'primaryFolder': 'All Locations',
   'secondaryFolders': None,
   'lastModified': '2020-06-18T10:15:45.4235115-06:0

In [3]:
#This is working. Just need to finish.
#This will produce step 1 of using the API.  Step two will be to take the output here and use the other API format using the uniqueID and the time start and end date.

# Disecting the nested list per nested list.  Input based on index of dataframe.

url = "https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/locations"
responseList = json.loads(requests.get(url).text)
SearchIndex = 0
outdfIndex = 0
datasetsIndex = 0

outdf = pd.DataFrame(columns=['locationName', 'identifier', 'loc_uniqueId', 'locationType', 'latitude', 'longitude', 'srid',
                              'ds_uniqueId', 'parameter', 'unit', 'rawStartTime', 'rawEndTime', 'label']) 
dfdatasets = pd.DataFrame(columns=['locationName', 'identifier', 'loc_uniqueId', 'locationType', 'latitude', 'longitude', 'srid',
                                   'ds_uniqueId', 'parameter', 'unit', 'rawStartTime', 'rawEndTime', 'label']) 


while SearchIndex < len(responseList):
    LtDi = responseList[SearchIndex]
    
    #locationName
    LtDi_lsn = LtDi['locationName']
    outdf.loc[outdfIndex, 'locationName'] = LtDi_lsn
    
    #list of identifier
    LtDi_is = LtDi['identifier']
    outdf.loc[outdfIndex, 'identifier'] = LtDi_is
    
    #list of loc_uniqueId
    LtDi_un = LtDi['uniqueId']
    outdf.loc[outdfIndex, 'loc_uniqueId'] = LtDi_un
    
    #list of locationType
    LtDi_lt = LtDi['locationType']
    outdf.loc[outdfIndex, 'locationType'] = LtDi_lt
    
    #list of latitude
    LtDi_la = LtDi['latitude']
    outdf.loc[outdfIndex, 'latitude'] = LtDi_la
    
    #list of longitude
    LtDi_lo = LtDi['longitude']
    outdf.loc[outdfIndex, 'longitude'] = LtDi_lo
    
    #list of srid
    LtDi_sr = LtDi['srid']
    outdf.loc[outdfIndex, 'srid'] = LtDi_sr
       
    # Time Series Info in New Nested List
    DL = LtDi['datasets']
    if DL is None:
        outdf.loc[outdfIndex, 'ds_uniqueId'] = ""
        outdf.loc[outdfIndex, 'parameter'] = ""
        outdf.loc[outdfIndex, 'unit'] = ""
        outdf.loc[outdfIndex, 'rawEndTime'] = ""
        outdf.loc[outdfIndex, 'rawStartTime'] = ""
        outdf.loc[outdfIndex, 'label'] = ""
    else:
        while datasetsIndex < len(DL):
            DLD = DL[datasetsIndex]
            
            #list of ds_uniqueId
            DLDstr = DLD['uniqueId']
            outdf.loc[outdfIndex, 'ds_uniqueId'] = DLDstr
            
            #list of parameter
            DLDstr = DLD['parameter']
            outdf.loc[outdfIndex, 'parameter'] = DLDstr
            
            #list of unit
            DLDstr = DLD['unit']
            outdf.loc[outdfIndex, 'unit'] = DLDstr
            
            #list of rawEndTime
            DLDstr = DLD['rawEndTime']
            outdf.loc[outdfIndex, 'rawEndTime'] = DLDstr
            
            #list of rawStartTime
            DLDstr = DLD['rawStartTime']
            outdf.loc[outdfIndex, 'rawStartTime'] = DLDstr
            
            #list of label
            DLDstr = DLD['label']
            outdf.loc[outdfIndex, 'label'] = DLDstr
            
            #Copy exiting rows from index row, repeat with index - datasetsIndex.
            outdf.loc[outdfIndex, 'locationName'] = outdf.loc[outdfIndex-datasetsIndex, 'locationName']
            outdf.loc[outdfIndex, 'identifier'] = outdf.loc[outdfIndex-datasetsIndex, 'identifier']
            outdf.loc[outdfIndex, 'loc_uniqueId'] = outdf.loc[outdfIndex-datasetsIndex, 'loc_uniqueId']
            outdf.loc[outdfIndex, 'locationType'] = outdf.loc[outdfIndex-datasetsIndex, 'locationType']
            outdf.loc[outdfIndex, 'latitude'] = outdf.loc[outdfIndex-datasetsIndex, 'latitude']
            outdf.loc[outdfIndex, 'longitude'] = outdf.loc[outdfIndex-datasetsIndex, 'longitude']
            outdf.loc[outdfIndex, 'srid'] = outdf.loc[outdfIndex-datasetsIndex, 'srid']
            datasetsIndex += 1  # Advanced datasetsIndex counter
            outdfIndex += 1 # Advanced outdfIndex counter due to extra rows if datasets is not None.
            
        datasetsIndex = 0  # reset datasetsIndex
    
    # Advanced Index Counter
    SearchIndex += 1
    outdfIndex += 1

print(len(outdf))
outdf.head(3)

370


,locationName,identifier,loc_uniqueId,locationType,latitude,longitude,srid,ds_uniqueId,parameter,unit,rawStartTime,rawEndTime,label
0,"Snake River nr Murphy, ID",13172500,9b3cea8425b54d019abf4082e7058b4c,Hydrology Station,0.0,0.0,4326,,,,,,
1,Swan Falls Dam,13172450,3d7ed60119b34a2d8a8f11a7973802a9,Hydrology Station,0.0,0.0,4326,,,,,,
2,Lower Salmon Falls Dam,13134995,b4cb7555831544d88db12ae5c4e4e7d5,Hydrology Station,0.0,0.0,4326,,,,,,


In [4]:
# Creating outputString for the url.

str1 = "https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/"

def assignoutputString(A, B, C):
    if A == '' or pd.isnull(A):
        outString = "Unspecified"
    else:
        outString = str1 + str(A) + "?fromDate=" + str(B) + "&toDate=" + str(C)
    return outString

outdf['outputString'] = outdf.apply(lambda row: assignoutputString(row['ds_uniqueId'], row['rawStartTime'], row['rawEndTime']), axis=1)
print(len(outdf))
outdf = outdf.reset_index(drop=True)
outdf.head(3)

370


,locationName,identifier,loc_uniqueId,locationType,latitude,longitude,srid,ds_uniqueId,parameter,unit,rawStartTime,rawEndTime,label,outputString
0,"Snake River nr Murphy, ID",13172500,9b3cea8425b54d019abf4082e7058b4c,Hydrology Station,0.0,0.0,4326,,,,,,,Unspecified
1,Swan Falls Dam,13172450,3d7ed60119b34a2d8a8f11a7973802a9,Hydrology Station,0.0,0.0,4326,,,,,,,Unspecified
2,Lower Salmon Falls Dam,13134995,b4cb7555831544d88db12ae5c4e4e7d5,Hydrology Station,0.0,0.0,4326,,,,,,,Unspecified


In [5]:
# Drop and only keep columns rows with "label" = Daily or daily.
outdf = outdf[(outdf.label == "Daily") | (outdf.label == "daily")]
print(len(outdf))
outdf = outdf.reset_index(drop=True)
outdf.head(3)

177


,locationName,identifier,loc_uniqueId,locationType,latitude,longitude,srid,ds_uniqueId,parameter,unit,rawStartTime,rawEndTime,label,outputString
0,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...
1,Upper Grand View Mutual,2003-1,88b8431b09f54d019edec6cb90715978,Hydrology Station,42.980944,-116.090639,4326,716bea6d967e432e8344b65e154d3bd7,Discharge,ft^3/s,2016-03-24T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...
2,Indian Cove Irrigation District,2004-T,b04258fd520c458f82785f10fe20e673,Hydrology Station,42.934833,-115.558778,4326,efbde743a92c44ce80b4bdbe46e64d54,Discharge,ft^3/s,2016-03-07T00:00:00-07:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...


In [6]:
# The TS API search.

tsdf = pd.DataFrame(columns=['locationName', 'loc_uniqueId',
                             'timeStamp', 'numericValue1']) 

tsSearchIndex = 0
outdfIndex = 0
tsdfIndex = 0

while outdfIndex < len(outdf):
    
    url = outdf.loc[outdfIndex, 'outputString']
    print(outdfIndex)
    print(url)
    
    if url == "Unspecified":
        # Copy exsisting rows from outdf to tsdf.
        tsdf.loc[tsdfIndex, "locationName"] = outdf.loc[outdfIndex, "locationName"]  
        tsdf.loc[tsdfIndex, "loc_uniqueId"] = outdf.loc[outdfIndex, "loc_uniqueId"] 
        tsdfIndex += 1
    else:
        #The URl
        resD = json.loads(requests.get(url).text)
        resDL = resD['points']
        print("Length is: ", len(resDL))
        
        tsSearchIndex = 0
        while tsSearchIndex < len(resDL):
            
            # Copy exsisting rows from outdf to tsdf.
            tsdf.loc[tsdfIndex, "locationName"] = outdf.loc[outdfIndex, "locationName"]  
            tsdf.loc[tsdfIndex, "loc_uniqueId"] = outdf.loc[outdfIndex, "loc_uniqueId"]
            
            #Time Series serach index.
            LtD = resDL[tsSearchIndex]
    
            #timeStamp
            LtD_ts = LtD['timeStamp']
            tsdf.loc[tsdfIndex, 'timeStamp'] = LtD_ts
            
            #numericValue1
            LtD_nv1 = LtD['numericValue1']
            tsdf.loc[tsdfIndex, 'numericValue1'] = LtD_nv1
            
            
#             print("outdfIndex is " + str(outdfIndex) + ", tsdfIndex is " + str(tsdfIndex) + ", tsSearchIndex is " + str(tsSearchIndex))
#             tsdflist = tsdf.loc[tsdfIndex].tolist()
#             print(tsdflist)
            
            tsdfIndex += 1
            tsSearchIndex += 1
    
    # Advanced Index Counter
    outdfIndex += 1

print(len(tsdf))
tsdf.head(3)

0
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/e78661a50ae8486ab51317993990370a?fromDate=2016-03-29T01:00:00-06:00&toDate=2022-03-25T01:00:00-06:00
Length is:  1881
1
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/716bea6d967e432e8344b65e154d3bd7?fromDate=2016-03-24T01:00:00-06:00&toDate=2022-03-25T01:00:00-06:00
Length is:  1986
2
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/efbde743a92c44ce80b4bdbe46e64d54?fromDate=2016-03-07T00:00:00-07:00&toDate=2022-03-25T01:00:00-06:00
Length is:  2036
3
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/ea3196f16f324af4a7f5a408e0a508f0?fromDate=2016-03-31T01:00:00-06:00&toDate=2022-03-25T01:00:00-06:00
Length is:  2022
4
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/dfc89958e5504aed9968c7a91d0e0aba?fromDate=2015-03-16T01:00:00-06:00&toDate=2022-03-01T00:00:00-07:00
Length is:  1529
5
https://research.idwr.idaho.gov/apps/hydrol

Length is:  1999
43
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/6c612b5748994e15a3d66352cbabd0e5?fromDate=2016-04-06T01:00:00-06:00&toDate=2022-03-25T01:00:00-06:00
Length is:  1299
44
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/db6a57e57b4c41d29becc1d9c04b224d?fromDate=2016-05-16T01:00:00-06:00&toDate=2022-03-25T01:00:00-06:00
Length is:  1464
45
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/96a1906efccd4111ba55771b107b20b8?fromDate=2016-04-22T01:00:00-06:00&toDate=2022-03-25T01:00:00-06:00
Length is:  1959
46
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/20990ab05dfc4c2491ccabd402bf2ab1?fromDate=2017-05-19T01:00:00-06:00&toDate=2022-03-26T01:00:00-06:00
Length is:  1556
47
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/ef9ad8467abf42eab683f31bfecce388?fromDate=2016-07-20T01:00:00-06:00&toDate=2022-03-25T01:00:00-06:00
Length is:  1434
48
https://research.idw

Length is:  5531
86
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/01050b6cf59249dda4cf33406b07e099?fromDate=2017-05-04T01:00:00-06:00&toDate=2021-10-29T01:00:00-06:00
Length is:  990
87
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/f8145df0fe6b42c8b7db1fd82bd9e6d7?fromDate=2006-01-01T00:00:00-07:00&toDate=2021-10-31T01:00:00-06:00
Length is:  5204
88
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/2c7833ad437b45dba42df78a27a5f6ae?fromDate=2004-12-02T00:00:00-07:00&toDate=2019-10-25T01:00:00-06:00
Length is:  4054
89
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/ea8838f5cae6464fafdc0b68c19e127d?fromDate=2005-01-01T00:00:00-07:00&toDate=2021-10-31T01:00:00-06:00
Length is:  5609
90
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/39723f3dfb5b467c81f3a33386fd0f9d?fromDate=2008-01-01T00:00:00-07:00&toDate=2021-10-19T01:00:00-06:00
Length is:  4421
91
https://research.idwr

Length is:  7219
129
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/6a0f85c4603f41ee89e2446290bdd0b2?fromDate=2002-03-27T00:00:00-07:00&toDate=2021-12-31T00:00:00-07:00
Length is:  7218
130
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/4d6f5b0bf2e24ae6b0585557a3c681e2?fromDate=2002-03-26T00:00:00-07:00&toDate=2021-12-31T00:00:00-07:00
Length is:  7219
131
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/7aa1ec1141c54bbdb3144178e2f861ba?fromDate=2002-03-27T00:00:00-07:00&toDate=2021-12-31T00:00:00-07:00
Length is:  7217
132
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/50de2ea95d08496da14de9cc9a5b1875?fromDate=2002-03-27T00:00:00-07:00&toDate=2021-12-31T00:00:00-07:00
Length is:  7219
133
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/eb8b1c70617a4b718de1a401d890217a?fromDate=2002-03-27T00:00:00-07:00&toDate=2021-12-31T00:00:00-07:00
Length is:  7140
134
https://resear

Length is:  3605
172
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/ac7908d2c6024a1695845377fa5d5c45?fromDate=2013-10-01T01:00:00-06:00&toDate=2016-11-03T01:00:00-06:00
Length is:  1066
173
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/858ae5de1eed4edb8c155ef8ceabae65?fromDate=2017-05-04T01:00:00-06:00&toDate=2021-10-31T01:00:00-06:00
Length is:  1008
174
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/0739caa6e3cc4ff0b47f5a87e3f3f20d?fromDate=2008-05-08T01:00:00-06:00&toDate=2021-10-18T01:00:00-06:00
Length is:  3210
175
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/6f72578fc47749499e0b14b67c8e5e68?fromDate=2020-10-01T01:00:00-06:00&toDate=2021-10-20T01:00:00-06:00
Length is:  371
176
https://research.idwr.idaho.gov/apps/hydrologic/aquainfo/api/telemetry/dc1d16651ccf47d5a6ad3987ff6638f8?fromDate=2020-10-01T01:00:00-06:00&toDate=2021-10-20T01:00:00-06:00
Length is:  278
568326


,locationName,loc_uniqueId,timeStamp,numericValue1
0,Snake River Irrigation District,364cf2cd6cfc41ae95456e496753fe9f,2016-03-29T01:00:00-06:00,181.932759
1,Snake River Irrigation District,364cf2cd6cfc41ae95456e496753fe9f,2016-03-30T01:00:00-06:00,190.773958
2,Snake River Irrigation District,364cf2cd6cfc41ae95456e496753fe9f,2016-03-31T01:00:00-06:00,198.166146


In [7]:
#Merge
outdf = pd.merge(outdf, tsdf, left_on='loc_uniqueId', right_on='loc_uniqueId', how='left')
print(len(outdf))
outdf.head(3)

568326


,locationName_x,identifier,loc_uniqueId,locationType,latitude,longitude,srid,ds_uniqueId,parameter,unit,rawStartTime,rawEndTime,label,outputString,locationName_y,timeStamp,numericValue1
0,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-29T01:00:00-06:00,181.932759
1,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-30T01:00:00-06:00,190.773958
2,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-31T01:00:00-06:00,198.166146


In [8]:
#Update datatype of date to fit WaDE 2.0 structure
outdf['timeStamp'] = pd.to_datetime(outdf['timeStamp'], utc=True)
outdf

,locationName_x,identifier,loc_uniqueId,locationType,latitude,longitude,srid,ds_uniqueId,parameter,unit,rawStartTime,rawEndTime,label,outputString,locationName_y,timeStamp,numericValue1
0,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-29 07:00:00+00:00,181.932759
1,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-30 07:00:00+00:00,190.773958
2,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-31 07:00:00+00:00,198.166146
3,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-04-01 07:00:00+00:00,198.171354
4,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-04-02 07:00:00+00:00,195.649479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568321,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-15 07:00:00+00:00,0.0
568322,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-16 07:00:00+00:00,0.0
568323,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-17 07:00:00+00:00,0.0
568324,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-18 07:00:00+00:00,0.0


In [9]:
#Extract year value for ReportYearCV
outdf['in_ReportYear'] = pd.DatetimeIndex(outdf['timeStamp']).year
outdf

,locationName_x,identifier,loc_uniqueId,locationType,latitude,longitude,srid,ds_uniqueId,parameter,unit,rawStartTime,rawEndTime,label,outputString,locationName_y,timeStamp,numericValue1,in_ReportYear
0,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-29 07:00:00+00:00,181.932759,2016
1,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-30 07:00:00+00:00,190.773958,2016
2,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-03-31 07:00:00+00:00,198.166146,2016
3,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-04-01 07:00:00+00:00,198.171354,2016
4,Snake River Irrigation District,2002-1,364cf2cd6cfc41ae95456e496753fe9f,Hydrology Station,42.950335,-115.974794,4326,e78661a50ae8486ab51317993990370a,Discharge,ft^3/s,2016-03-29T01:00:00-06:00,2022-03-25T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Snake River Irrigation District,2016-04-02 07:00:00+00:00,195.649479,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568321,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-15 07:00:00+00:00,0.0,2021
568322,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-16 07:00:00+00:00,0.0,2021
568323,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-17 07:00:00+00:00,0.0,2021
568324,Hawley Creek Lower,444019113180701,0e7c7f9891224f119ed72c31f44aaefa,Hydrology Station,44.671965,-113.301801,4326,dc1d16651ccf47d5a6ad3987ff6638f8,Discharge,ft^3/s,2020-10-01T01:00:00-06:00,2021-10-20T01:00:00-06:00,Daily,https://research.idwr.idaho.gov/apps/hydrologi...,Hawley Creek Lower,2021-10-18 07:00:00+00:00,0.0,2021


In [10]:
#Exporting to Finished File
outdf.to_csv('P_idOSMaster.csv', index=False)  # The output